## Supporting Information.
Borofsky, T. and Feldman, M. 2020. Static environments with limited resources select for multiple foraging strategies rather than conformity in social learners.
# Appendix S2: Solving for Initial Equilibria

In [2]:
#from PIL import Image
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages')
#import os
import numpy as np
import importlib
from numpy import linalg
import PO_helperfuns
from PO_helperfuns import *
import PO_DatFrameFuns
from PO_DatFrameFuns import *
import scipy.stats as scs
import matplotlib as mpl
import matplotlib.pyplot as plt
#importlib.reload(helperfuns)
import pandas as pd
import sympy as sp
from sympy import *
from sympy.solvers import solve
#np.set_printoptions(precision=3, suppress = True)
#import seaborn as sns
# next two libraries are used to flatten list of lists
import functools
import operator
# for formatting tick labels
#from matplotlib.ticker import FormatStrFormatter

#for parallelizing:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool




## First we set up a parameter mesh, as described in Section 5.1

In [2]:

svals = np.array([0,0.5,1,3])
muvals = np.array([-2, -0.5, 0,0.5,2])
betavals = np.array([0,0.33, 0.66, 1])
df = get_param_grid(svals, muvals, betavals)

## We use parallelization to to greatly speed up the dataframe generation. 

In [3]:
# Parallelize and find equilibria

cores=mp.cpu_count()
# split dataframe over cores, stored in df_split
df_split = np.array_split(df, cores, axis=0)

# remove this line... just for testing
pool = Pool(cores)

# Run get_1side_df over each dataframe in df_split. 
# get_1side_df iterates 50000 time steps. Stack dataframe (df) parts into one df.

df_out = np.vstack(pool.map(get_1side_df, df_split))
pool.close()
pool.join()
pool.clear()
df_1side = pd.DataFrame(df_out, columns = df.columns)

# fix the column names
colnames = ['mu', 'K', 'pc', 's', 'beta',
       'u1init', 'u2init', 'buinit', 'r1init', 'r2init', 'u1eq', 'u2eq',
       'bueq', 'r1eq', 'r2eq', 'Weq', 'time', 'reached_eq', 'URstable']
df_1side = df_1side[colnames] # gets rid of the added unnamed columns
#save to csv
df_1side.to_csv('data_1side_1stRound.csv')

## Check the result of the 500,000th iteration to see if it is at equilibrium.

In [98]:
df_1side = pd.read_csv('data_1side_1stRound.csv')

In [4]:

u1 = df_1side.u1eq; beta = df_1side.beta; pc = df_1side.pc; K = df_1side.K
should_be_0 = u1**2*2*beta*(K*(1/2)+pc) - u1*(1+pc+(K*(1/2)+pc)*(2+beta)) +2*(K*(1/2)+pc)

In [5]:
where_wrong = np.abs(should_be_0)>1e-10

In [6]:
should_be_0[where_wrong]

225     0.359054
230    0.0544017
235     0.214528
240     0.477529
245    -0.230163
250     0.149606
255    -0.199474
260   -0.0374013
265     0.344226
270    -0.411416
275     0.547493
280      0.44399
285     0.505728
290     0.574984
295     0.256465
dtype: object

In [7]:
df_1side[where_wrong]

,mu,K,pc,s,beta,u1init,u2init,buinit,r1init,r2init,u1eq,u2eq,bueq,r1eq,r2eq,Weq,time,reached_eq,URstable
225,-2,0.841344,2.86652e-07,3,0,0.05,0.1,0.85,0.05,0.1,0.261923,0.651914,0.0861627,1,1,1.84135,500000,False,0
230,-2,0.841344,2.86652e-07,3,0,0.05,0.1,0.85,0.5,0.3,0.427374,0.486463,0.0861627,1,1,1.84135,500000,False,0
235,-2,0.841344,2.86652e-07,3,0,0.05,0.1,0.85,0.85,0.9,0.340412,0.573425,0.0861627,1,1,1.84135,500000,False,0
240,-2,0.841344,2.86652e-07,3,0,0.05,0.1,0.85,0.2,0.89,0.197582,0.716256,0.0861627,1,1,1.84135,500000,False,0
245,-2,0.841344,2.86652e-07,3,0,0.05,0.1,0.85,0.9,0.2,0.581916,0.331922,0.0861627,1,1,1.84135,500000,False,0
250,-2,0.841344,2.86652e-07,3,0,0.48,0.4,0.12,0.05,0.1,0.37567,0.538167,0.0861627,1,1,1.84135,500000,False,0
255,-2,0.841344,2.86652e-07,3,0,0.48,0.4,0.12,0.5,0.3,0.565249,0.348588,0.0861627,1,1,1.84135,500000,False,0
260,-2,0.841344,2.86652e-07,3,0,0.48,0.4,0.12,0.85,0.9,0.477231,0.436607,0.0861627,1,1,1.84135,500000,False,0
265,-2,0.841344,2.86652e-07,3,0,0.48,0.4,0.12,0.2,0.89,0.269976,0.643861,0.0861627,1,1,1.84135,500000,False,0
270,-2,0.841344,2.86652e-07,3,0,0.48,0.4,0.12,0.9,0.2,0.680351,0.233487,0.0861627,1,1,1.84135,500000,False,0


# Reflect the dataframe

In [108]:
# note: once i re-run with the fixed check of if it reached eq, this cell won't be necessary
df_1side.loc[~where_wrong, 'reached_eq'] = True
df_1side.loc[where_wrong,'reached_eq'] = False


In [8]:

df_total = reflect_df(df_1side)
df = df_total.sort_values(by=['mu','beta','s'])
df.reset_index(inplace=True, drop=True) 
colnames = ['mu', 'K', 'pc', 's', 'beta',
       'u1init', 'u2init', 'buinit', 'r1init', 'r2init', 'u1eq', 'u2eq',
       'bueq', 'r1eq', 'r2eq', 'Weq', 'time', 'reached_eq', 'URstable']
df = df[colnames]
df.to_csv('data.csv')

# Find unique equilibria and analyze them

In [2]:
# We have a lot of repeat equilibria in the dataframe df because many of the initial points go to the same equilibrium.
# We now extract the unique equilibria for each parameter combination.
df = pd.read_csv('data.csv')
df_main = df[(df.reached_eq==1)]
unique_eq = get_UniqueEquilibria(df_main)
diff = np.abs(unique_eq.u1eq - unique_eq.u2eq)
unique_eq['difference'] = diff
unique_eq.to_csv('UniqueEquilibriaDF.csv')

In [5]:
unique_eq.query('C_s > 0')

,K,pc,s,mu,beta,u1eq,u2eq,bueq,r1eq,r2eq,Weq,URstable,NumInitials,C_s,difference
20,0.060598,0.006210,0.5,-2.0,0.00,0.067657,0.067657,0.864687,1.000000,1.000000,1.079227,0.0,30,0.087527,0.0
21,0.060598,0.006210,0.5,-2.0,0.33,0.067009,0.067009,0.865982,0.977887,0.977887,1.077612,0.0,30,0.085360,0.0
22,0.060598,0.006210,0.5,-2.0,0.66,0.066372,0.066372,0.867257,0.956195,0.956195,1.076028,0.0,30,0.083228,0.0
23,0.060598,0.006210,0.5,-2.0,1.00,0.065726,0.065726,0.868549,0.934274,0.934274,1.074427,0.0,30,0.081067,0.0
28,0.157305,0.001350,1.0,-2.0,0.00,0.137775,0.137775,0.724451,1.000000,1.000000,1.161355,0.0,20,0.200720,0.0
29,0.157305,0.001350,1.0,-2.0,0.33,0.135529,0.135529,0.728943,0.955276,0.955276,1.154199,0.0,30,0.192760,0.0
30,0.157305,0.001350,1.0,-2.0,0.66,0.133329,0.133329,0.733342,0.912003,0.912003,1.147275,0.0,30,0.184964,0.0
31,0.157305,0.001350,1.0,-2.0,1.00,0.131111,0.131111,0.737778,0.868889,0.868889,1.140377,0.0,30,0.177102,0.0
48,0.624655,0.066807,1.0,-0.5,0.00,0.415473,0.415473,0.169055,1.000000,1.000000,1.825077,0.0,20,0.050973,0.0
49,0.624655,0.066807,1.0,-0.5,0.33,0.410433,0.410433,0.179135,0.864557,0.864557,1.722375,0.0,30,0.036512,0.0


# Check Stability

It helps to have columns documenting which alleles can invade. These columns are x_pos_invades, x_neg_invades, D_pos_invades, and D_neg_invades:
- x_pos_invades = TRUE if $C_s > 0$ and $K < 1$. Else false
- x_neg_invades = TRUE if $C_s < 0$ and $K > 0$. Else false
- D_pos_invades = TRUE if $C_D > 0$ and $D < 1$. Else false
- D_neg_invades = TRUE if $C_D < 0$ and $D > -2$  
There are many cases of $C_D = 0$ that need to be resolved. There are no cases of $C_s = 0$.

In [4]:
unique_eq['x_pos_invades'] = unique_eq.C_s > 0


unique_eq['x_neg_invades'] = unique_eq.C_s < 0
unique_eq.loc[unique_eq.K == 0, 'x_neg_invades'] = False

unique_eq['y_pos_invades'] = unique_eq.C_D > 0
unique_eq.loc[unique_eq.D == 1, 'y_pos_invades'] = False

unique_eq['y_neg_invades'] = unique_eq.C_D < 0
unique_eq.loc[unique_eq.D == -2, 'y_neg_invades'] = False

unique_eq.to_csv('UniqueEquilibriaDF.csv')

In [8]:
unique_eq = pd.read_csv('UniqueEquilibriaDF.csv')


In [9]:
#unique_eq = pd.read_csv('UniqueEquilibriaDF.csv')

cores=mp.cpu_count()
df_split = np.array_split(unique_eq, cores, axis=0)
pool = Pool(cores)
df_out = np.vstack(pool.map(df_ext_stability_iterate, df_split))
pool.close()
pool.join()
pool.clear()


In [10]:
cols =  unique_eq.columns
df_fixed = pd.DataFrame(df_out, columns = cols)

#df_fixed.to_csv('UniqueEquilibriaDF_fixed.csv')

In [11]:
df_fixed

,Unnamed: 0,K,pc,s,mu,beta,u1eq,u2eq,bueq,r1eq,r2eq,Weq,URstable,NumInitials,C_s,difference,x_pos_invades,x_neg_invades
0,0,0.0,0.02275,0.0,-2.0,0.0,0.042593,0.042593,0.914813,1.0,1.0,1.06825,0.0,30,-0.050542,0.0,False,False
1,1,0.0,0.02275,0.0,-2.0,0.33,0.042321,0.042321,0.915358,0.986034,0.986034,1.067615,0.0,30,-0.050572,0.0,False,False
2,2,0.0,0.02275,0.0,-2.0,0.66,0.042052,0.042052,0.915896,0.972246,0.972246,1.066988,0.0,30,-0.050601,0.0,False,False
3,3,0.0,0.02275,0.0,-2.0,1.0,0.041778,0.041778,0.916444,0.958222,0.958222,1.066349,0.0,30,-0.050632,0.0,False,False
4,4,0.0,0.308538,0.0,-0.5,0.0,0.320456,0.320456,0.359087,1.0,1.0,1.925613,0.0,30,-0.182833,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,70,0.993558,0.00621,3.0,0.5,0.66,0.499931,0.499931,0.000138,0.670046,0.670046,1.68026,0.0,30,-0.017074,0.0,False,True
71,71,0.993558,0.00621,3.0,0.5,1.0,0.499923,0.499923,0.000154,0.500077,0.500077,1.509276,0.0,30,-0.017132,0.0,False,True
72,72,0.9973,0.00135,3.0,0.0,0.33,0.499632,0.499632,0.000735,0.835121,0.835121,1.836471,0.0,30,-0.002413,0.0,False,True
73,73,0.9973,0.00135,3.0,0.0,0.66,0.499596,0.499596,0.000808,0.670266,0.670266,1.671616,0.0,30,-0.002651,0.0,False,True


Now we check that the values of $C_s$ reflect whether invasion actually happens.

In [17]:
# find examples that don't agree with the sign of C_s
df_disagree_x_neg = df_fixed.query('C_s < 0 & x_neg_invades == False')
#max(np.abs(df_disagree_x_neg.C_s))

In [20]:
df_disagree_x_neg

,Unnamed: 0,K,pc,s,mu,beta,u1eq,u2eq,bueq,r1eq,r2eq,Weq,URstable,NumInitials,C_s,difference,x_pos_invades,x_neg_invades
0,0,0.0,0.02275,0.0,-2.0,0.0,0.042593,0.042593,0.914813,1.0,1.0,1.06825,0.0,30,-0.050542,0.0,False,False
1,1,0.0,0.02275,0.0,-2.0,0.33,0.042321,0.042321,0.915358,0.986034,0.986034,1.067615,0.0,30,-0.050572,0.0,False,False
2,2,0.0,0.02275,0.0,-2.0,0.66,0.042052,0.042052,0.915896,0.972246,0.972246,1.066988,0.0,30,-0.050601,0.0,False,False
3,3,0.0,0.02275,0.0,-2.0,1.0,0.041778,0.041778,0.916444,0.958222,0.958222,1.066349,0.0,30,-0.050632,0.0,False,False
4,4,0.0,0.308538,0.0,-0.5,0.0,0.320456,0.320456,0.359087,1.0,1.0,1.925613,0.0,30,-0.182833,0.0,False,False
5,5,0.0,0.308538,0.0,-0.5,0.33,0.314284,0.314284,0.371432,0.896286,0.896286,1.861613,0.0,30,-0.189118,0.0,False,False
6,6,0.0,0.308538,0.0,-0.5,0.66,0.307948,0.307948,0.384104,0.796754,0.796754,1.800195,0.0,30,-0.195571,0.0,False,False
7,7,0.0,0.308538,0.0,-0.5,1.0,0.30127,0.30127,0.397459,0.69873,0.69873,1.739706,0.0,30,-0.202371,0.0,False,False
8,8,0.0,0.5,0.0,0.0,0.0,0.4,0.4,0.2,1.0,1.0,2.5,0.0,30,-0.159577,0.0,False,False
9,9,0.0,0.5,0.0,0.0,0.33,0.394506,0.394506,0.210987,0.869813,0.869813,2.369813,0.0,30,-0.168343,0.0,False,False


We confirmed that the equation to find u1 = u2 equilibria works. Now we want to find parameter values for which the equilibrium is unstable or there are cycles (complex eigenvalues)

In [13]:
A,B,C,D,E = sp.symbols('a, b, c, d, e')
J = Matrix([[A,-A,B,C],[-A,A,C,B],[D,0,E,0],[0,D,0,E]])

charpoly = J.charpoly(lamda).as_expr()

collect(charpoly,lamda)

-2*a*b*d*e - 2*a*c*d*e + b**2*d**2 - c**2*d**2 + lamda**4 + lamda**3*(-2*a - 2*e) + lamda**2*(4*a*e - 2*b*d + e**2) + lamda*(2*a*b*d + 2*a*c*d - 2*a*e**2 + 2*b*d*e)

In [14]:
factor(charpoly)

-(b*d + c*d + e*lamda - lamda**2)*(2*a*e - 2*a*lamda - b*d + c*d - e*lamda + lamda**2)

In [3]:
u1, r1, K, pc, L,R, W = sp.symbols('\hat{u}_1, \hat{r}_1,  K, \pi_C, L(1/2),R, \hat{W}')
from sympy.abc import beta, lamda
# R = 1 + r1
#W = 1 + pc + 2*r1*L
A = K*R/(4*W*u1)
B = L*(1-u1)
C = -u1*L
D = -beta*r1/R
E = 1/R

J = Matrix([[A,-A,B,C],[-A,A,C,B],[D,0,E,0],[0,D,0,E]])

In [4]:
charpoly = factor(J.charpoly(lamda).as_expr())
charpoly

(2*L(1/2)*\hat{r}_1*\hat{u}_1*beta - L(1/2)*\hat{r}_1*beta - R*lamda**2 + lamda)*(K*R**2*lamda - K*R - 2*L(1/2)*\hat{W}*\hat{r}_1*\hat{u}_1*beta - 2*R*\hat{W}*\hat{u}_1*lamda**2 + 2*\hat{W}*\hat{u}_1*lamda)/(2*R**2*\hat{W}*\hat{u}_1)

In [5]:
charpoly

(2*L(1/2)*\hat{r}_1*\hat{u}_1*beta - L(1/2)*\hat{r}_1*beta - R*lamda**2 + lamda)*(K*R**2*lamda - K*R - 2*L(1/2)*\hat{W}*\hat{r}_1*\hat{u}_1*beta - 2*R*\hat{W}*\hat{u}_1*lamda**2 + 2*\hat{W}*\hat{u}_1*lamda)/(2*R**2*\hat{W}*\hat{u}_1)

In [6]:
n,d = fraction(charpoly)
charpoly = n
print(latex(charpoly))

\left(2 L(1/2) \hat{r}_1 \hat{u}_1 \beta - L(1/2) \hat{r}_1 \beta - R \lambda^{2} + \lambda\right) \left(K R^{2} \lambda - K R - 2 L(1/2) \hat{W} \hat{r}_1 \hat{u}_1 \beta - 2 R \hat{W} \hat{u}_1 \lambda^{2} + 2 \hat{W} \hat{u}_1 \lambda\right)


In [92]:
evals = J.eigenvals(lamda)
evals = list(evals)

In [97]:
evals[2]

(K*R**2 + 2*\hat{W}*\hat{u}_1 + sqrt(K**2*R**4 - 4*K*R**2*\hat{W}*\hat{u}_1 - 16*L(1/2)*R*\hat{W}**2*\hat{r}_1*\hat{u}_1**2*beta + 4*\hat{W}**2*\hat{u}_1**2))/(4*R*\hat{W}*\hat{u}_1)

In [96]:
def Jac_UR_evals(u1, r1,K, pc, beta):
    R = 1 + r1
    L = K/2+pc
    W = 1 + pc +2*r1*L
    
    # constants
    L = K/2 + pc
    xi = K*(1+r1)/(4*u1**2)
    W = 1 + pc + 2*r1*(K/2 + pc)
    row1 = np.array([L*(1-u1)/W, -u1*L/W, u1*xi/W, -u1*xi/W])
    row2 = np.array([-u1*L/W, L*(1-u1)/W, -u1*xi/W, u1*xi/W])
    row3 = np.array([-beta*r1/(1+r1), 0, 1/(1+r1),0])
    row4 = np.array([0, -beta*r1/(1+r1), 0, 1/(1+r1)])
    Jac = np.array([row1, row2, row3, row4])
    return(Jac)



In [71]:
# parameter values
muvals = np.arange(-1,1.2,0.2)
svals = np.arange(0,4.2,0.2)
betavals = np.arange(0,1.1,0.1)

# make mesh
mumesh, smesh,betamesh = np.meshgrid(muvals,svals,betavals)

# flatten mesh
muvec, svec, betavec = [np.ndarray.flatten(item) for item in [mumesh,smesh,betamesh]]

# find K and pc values
norms = scs.norm(muvec,1)
Kvec = Kfun(svec,norms)
pcvec = pcfun(svec,norms)

# find equilibria
u1vec = PredictEquilibrium_NoPref(Kvec,pcvec,betavec)
r1vec = 1 - betavec*u1vec


# find the Jacobian and then find eigenvalues
Jacs = [Jac_UR_evals(u1,r1,K,pc,beta) for u1,r1,K,pc,beta in zip(u1vec, r1vec, Kvec, pcvec, betavec)]
evals1 = np.array([LA.eigvals(Jac)for Jac in Jacs])

# use my equations for the eigenvalues
evals2 = lambdastar(u1,r1, K,pc,beta)
evals2 = np.transpose(evals2)

# they're the same! it works!

IsComplex = np.iscomplex(evals1)

moduli = np.sqrt(np.real(evals1)**2 + np.imag(evals1)**2)
print(sum(moduli>=1))
# all are stable

complexrows = np.sum(IsComplex,1)>0
K_complex = Kvec[complexrows]
pc_complex = pcvec[complexrows]
beta_complex = betavec[complexrows]

[0 0 0 0]


In [76]:
len(Kvec)

2541

In [72]:
beta_complex

array([0.3, 0.4, 0.5, ..., 0.8, 0.9, 1. ])

In [73]:
K_complex

array([0.09678573, 0.09678573, 0.09678573, ..., 0.99864982, 0.99864982,
       0.99864982])

In [74]:
pc_complex

array([0.11506967, 0.11506967, 0.11506967, ..., 0.0013499 , 0.0013499 ,
       0.0013499 ])

In [75]:
pd.dataframe(data={'pc':pc_complex, 'K':K_complex, 'beta':beta_complex})

AttributeError: module 'pandas' has no attribute 'dataframe'